<a href="https://colab.research.google.com/github/lilphonsdale/Home_Sales/blob/main/Home_Sales_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.3.1'
spark_version = 'spark-3.4.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Hit:4 http://archive.ubuntu.com/ubuntu focal InRelease
Get:5 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Get:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease [18.1 kB]
Get:7 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Hit:8 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:9 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Get:10 http://security.ubuntu.com/ubuntu focal-security/main amd64 Packages [2,681 kB]
Get:11 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease [24.3 kB]
Get:12 http://security.ubuntu.com/ubuntu focal-security/universe amd64 Packages [1,046 kB]
Hit:13 http://ppa.launchpad.net/ubuntugis/ppa/ubuntu focal InRe

In [2]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [7]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), sep=",", header=True)

In [9]:
df.summary().show()

+-------+--------------------+----------+------------------+------------------+------------------+-----------------+------------------+------------------+------------------+-------------------+------------------+
|summary|                  id|      date|        date_built|             price|          bedrooms|        bathrooms|       sqft_living|          sqft_lot|            floors|         waterfront|              view|
+-------+--------------------+----------+------------------+------------------+------------------+-----------------+------------------+------------------+------------------+-------------------+------------------+
|  count|               33287|     33287|             33287|             33287|             33287|            33287|             33287|             33287|             33287|              33287|             33287|
|   mean|                null|      null|2013.5174091987863|314907.81797698804| 3.082224291765554|2.588758374140055|2116.5057529966653|11519.8677261

In [8]:
# 2. Create a temporary view of the DataFrame.
df.createOrReplaceTempView('homesales')

In [10]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?
spark.sql("""select round(avg(price),2) as AveragePrice from homesales where bedrooms=4""").show()


+------------+
|AveragePrice|
+------------+
|   299661.01|
+------------+



In [15]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?
spark.sql("""select date_built, round(avg(price),2) as AveragePrice from homesales where bathrooms=3 and bedrooms=3 group by date_built order by date_built asc""").show()


+----------+------------+
|date_built|AveragePrice|
+----------+------------+
|      2010|   292859.62|
|      2011|   291117.47|
|      2012|   293683.19|
|      2013|   295962.27|
|      2014|   290852.27|
|      2015|    288770.3|
|      2016|   290555.07|
|      2017|   292676.79|
+----------+------------+



In [17]:
# 5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?
spark.sql("""
select date_built, 
round(avg(price),2) as AveragePrice 
from homesales 
where bathrooms=3 
and bedrooms=3 
and floors=2
and sqft_living>=2000
group by date_built 
order by date_built asc""").show()



+----------+------------+
|date_built|AveragePrice|
+----------+------------+
|      2010|   285010.22|
|      2011|   276553.81|
|      2012|   307539.97|
|      2013|   303676.79|
|      2014|   298264.72|
|      2015|   297609.97|
|      2016|    293965.1|
|      2017|   280317.58|
+----------+------------+



In [36]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.

start_time = time.time()

spark.sql("""
select 
round(avg(view ),2) as View,
round(avg(price),2) as AveragePrice 
from homesales 
where price>=350000 
""").show()

print("--- %s seconds ---" % (time.time() - start_time))

+-----+------------+
| View|AveragePrice|
+-----+------------+
|32.26|   473796.26|
+-----+------------+

--- 0.6384458541870117 seconds ---


In [33]:
# 7. Cache the the temporary table home_sales.
spark.sql("cache table homesales")

DataFrame[]

In [34]:
# 8. Check if the table is cached.
spark.catalog.isCached('homesales')

True

In [37]:
# 9. Using the cached data, run the query that filters out the view ratings with average price 
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.

start_time = time.time()

spark.sql("""
select 
round(avg(view ),2) as View,
round(avg(price),2) as AveragePrice 
from homesales 
where price>=350000 
""").show()

print("--- %s seconds ---" % (time.time() - start_time))


+-----+------------+
| View|AveragePrice|
+-----+------------+
|32.26|   473796.26|
+-----+------------+

--- 0.25881505012512207 seconds ---


In [38]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data 
df.write.partitionBy("date_built").mode("overwrite").parquet("homesales_partitioned")

In [39]:
# 11. Read the parquet formatted data.
p_df_p=spark.read.parquet('homesales_partitioned')

In [40]:
# 12. Create a temporary table for the parquet data.
p_df_p.createOrReplaceTempView('p_homesales_p')

In [41]:
# 13. Run the query that filters out the view ratings with average price of greater than or equal to $350,000 
# with the parquet DataFrame. Round your average to two decimal places. 
# Determine the runtime and compare it to the cached version.

start_time = time.time()

spark.sql("""
select 
round(avg(view ),2) as View,
round(avg(price),2) as AveragePrice 
from homesales 
where price>=350000 
""").show()

print("--- %s seconds ---" % (time.time() - start_time))

+-----+------------+
| View|AveragePrice|
+-----+------------+
|32.26|   473796.26|
+-----+------------+

--- 0.27688121795654297 seconds ---


In [42]:
# 14. Uncache the home_sales temporary table.
spark.sql("uncache table homesales")

DataFrame[]

In [43]:
# 15. Check if the home_sales is no longer cached

spark.catalog.isCached('homesales')

False